# .to(device)

In [2]:
import torch

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
device

'cuda'

In [5]:
a = torch.randn(10)
b = a.to(device)

In [6]:
# aはデフォルトのcpu上にある
a

tensor([ 0.0973,  1.8751, -1.7213,  1.2197,  1.6355, -2.2699,  1.4848, -1.5587,
         0.7017,  0.4882])

In [8]:
# bはcudaに移動させたのでGPU上にある
b

tensor([ 0.0973,  1.8751, -1.7213,  1.2197,  1.6355, -2.2699,  1.4848, -1.5587,
         0.7017,  0.4882], device='cuda:0')

In [9]:
# デバイスが異なるので演算できない
a+b

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [10]:
import torch
import time

# CPU上でTensorを作成
tensor_cpu = torch.randn(10000, 10000)

# GPU上でTensorを作成 (もしGPUが利用可能なら)
device = "cuda" if torch.cuda.is_available() else "cpu"
tensor_gpu = tensor_cpu.to(device)

# CPU上での計算の時間を測定
start_time = time.time()
result_cpu = torch.mm(tensor_cpu, tensor_cpu)
end_time = time.time()
print(f"Time taken on CPU: {end_time - start_time:.5f} seconds")

# GPU上での計算の時間を測定 (もしGPUが利用可能なら)
if device == "cuda":
    start_time = time.time()
    result_gpu = torch.mm(tensor_gpu, tensor_gpu)
    end_time = time.time()
    print(f"Time taken on GPU: {end_time - start_time:.5f} seconds")

Time taken on CPU: 16.40714 seconds
Time taken on GPU: 0.11094 seconds


# MNISTのコードをGPUで実行

In [11]:
import torch
from sklearn import datasets
import matplotlib.pyplot as plt
from torch.nn import functional as F
from sklearn.model_selection import train_test_split
import numpy as np

## CPUで計算

In [18]:
# ======モデル======
class Linear():
    def __init__(self, in_features, out_features):
        self.W = torch.randn((out_features, in_features)) * torch.sqrt(torch.tensor(2.0 / in_features))
        self.W.requires_grad = True
        self.b = torch.zeros((1, out_features), requires_grad=True)

    def forward(self, X):
        self.X = X
        self.Z = X @ self.W.T + self.b
        return self.Z

    def backward(self, Z):
        self.W.grad_ = Z.grad_.T @ self.X
        self.b.grad_ = torch.sum(Z.grad_, dim=0)
        self.X.grad_ = Z.grad_ @ self.W
        return self.X.grad_

class ReLU():
    def forward(self, X):
        self.X = X
        return X.clamp_min(0.)

    def backward(self, A):
        return A.grad_ * (self.X > 0).float()

class SoftmaxCrossEntropy:
    def forward(self, X, y):
        e_x = torch.exp(X - torch.max(X, dim=-1, keepdim=True)[0])
        self.softmax_out = e_x / (torch.sum(e_x, dim=-1, keepdim=True) + 1e-10)

        log_probs = torch.log(self.softmax_out + 1e-10)
        target_log_probs = log_probs * y

        self.loss = -target_log_probs.sum(dim=-1).mean()
        return self.loss

    def backward(self, y):
        return (self.softmax_out - y) / y.shape[0]

class Model:
    def __init__(self, input_features, hidden_units, output_units):
        self.linear1 = Linear(input_features, hidden_units)
        self.relu = ReLU()
        self.linear2 = Linear(hidden_units, output_units)
        self.loss_fn = SoftmaxCrossEntropy()

    def forward(self, X, y):
        self.X = X
        self.Z1 = self.linear1.forward(X)
        self.A1 = self.relu.forward(self.Z1)
        self.Z2 = self.linear2.forward(self.A1)
        self.loss = self.loss_fn.forward(self.Z2, y)
        return self.loss, self.Z2

    def backward(self, y):
        self.Z2.grad_ = self.loss_fn.backward(y)
        self.A1.grad_ = self.linear2.backward(self.Z2)
        self.Z1.grad_ = self.relu.backward(self.A1)
        self.X.grad_ = self.linear1.backward(self.Z1)

    def zero_grad(self):
        # 勾配の初期化
        self.linear1.W.grad_ = None
        self.linear1.b.grad_ = None
        self.linear2.W.grad_ = None
        self.linear2.b.grad_ = None

    def step(self, learning_rate):
        # パラメータの更新
        self.linear1.W -= learning_rate * self.linear1.W.grad_
        self.linear1.b -= learning_rate * self.linear1.b.grad_
        self.linear2.W -= learning_rate * self.linear2.W.grad_
        self.linear2.b -= learning_rate * self.linear2.b.grad_

## Refactoring後の学習ループ(OptimizerやDataset, Dataloaderは後ほどRefactoring)
# ===データの準備====
dataset = datasets.load_digits()
data = dataset['data']
target = dataset['target']
images = dataset['images']
X_train, X_val, y_train, y_val = train_test_split(images, target, test_size=0.2, random_state=42)
X_train = (X_train - X_train.mean()) / X_train.std()
X_val = (X_val - X_train.mean()) / X_train.std()
X_train = torch.tensor(X_train.reshape(-1, 64), dtype=torch.float32)
X_val = torch.tensor(X_val.reshape(-1, 64), dtype=torch.float32)
y_train = F.one_hot(torch.tensor(y_train), num_classes=10) #1437 x 10
y_val = F.one_hot(torch.tensor(y_val), num_classes=10) # 360 x 10
batch_size = 30
# モデルの初期化
model = Model(input_features=64, hidden_units=10, output_units=10)

learning_rate = 0.01

# ログ
train_losses = []
val_losses = []
val_accuracies = []
for epoch in range(100):
    # エポック毎にデータをシャッフル
    shuffled_indices = np.random.permutation(len(y_train))
    num_batches = np.ceil(len(y_train)/batch_size).astype(int)
    running_loss = 0.0
    start_time = time.time()

    for i in range(num_batches):

        # mini batch作成
        start = i * batch_size
        end = start + batch_size

        batch_indices = shuffled_indices[start:end]
        y_true_ = y_train[batch_indices, :] # batch_size x 10

        X = X_train[batch_indices] # batch_size x 64
        # 順伝播と逆伝播の計算
        loss, _ = model.forward(X, y_true_)
        model.backward(y_true_)
        running_loss += loss.item()

        # パラメータ更新
        with torch.no_grad():
            model.step(learning_rate)

        model.zero_grad()

    # validation
    with torch.no_grad():
        val_loss, Z2_val = model.forward(X_val, y_val)

        val_accuracy = torch.sum(torch.argmax(Z2_val, dim=-1) == torch.argmax(y_val, dim=-1)) / y_val.shape[0]

    train_losses.append(running_loss/num_batches)
    val_losses.append(val_loss.item())
    val_accuracies.append(val_accuracy)
    end_time = time.time()
    epoch_duration = end_time - start_time
    print(f'epoch: {epoch}: train error: {running_loss/num_batches}, validation error: {val_loss.item()}, validation accuracy: {val_accuracy}, epoch duration {epoch_duration:.2f} sec')

epoch: 0: train error: 2.5243888745705285, validation error: 14.812721252441406, validation accuracy: 0.0833333358168602, epoch duration 0.03 sec
epoch: 1: train error: 2.2210290282964706, validation error: 14.17820930480957, validation accuracy: 0.08888889104127884, epoch duration 0.03 sec
epoch: 2: train error: 2.0964277957876525, validation error: 13.571051597595215, validation accuracy: 0.10000000149011612, epoch duration 0.04 sec
epoch: 3: train error: 1.9811919778585434, validation error: 12.899039268493652, validation accuracy: 0.11944444477558136, epoch duration 0.04 sec
epoch: 4: train error: 1.8539886871973674, validation error: 12.348795890808105, validation accuracy: 0.17222222685813904, epoch duration 0.04 sec
epoch: 5: train error: 1.7089916691184044, validation error: 11.778763771057129, validation accuracy: 0.19166666269302368, epoch duration 0.05 sec
epoch: 6: train error: 1.5694288884600003, validation error: 11.004616737365723, validation accuracy: 0.2027777731418609

## GPUで計算

In [19]:
device = 'cuda'
# ======モデル======
class Linear():
    def __init__(self, in_features, out_features, n):
        # ======================= Wとbのtensorを.to(device)する =================================
        self.W = (torch.randn((out_features, in_features)) * torch.sqrt(torch.tensor(2.0 / n))).to(device)
        self.W.requires_grad = True
        self.b = torch.zeros((1, out_features)).to(device)
        self.b.requires_grad = True

    def forward(self, X):
        self.X = X
        self.Z = X @ self.W.T + self.b
        return self.Z

    def backward(self, Z):
        self.W.grad_ = Z.grad_.T @ self.X
        self.b.grad_ = torch.sum(Z.grad_, dim=0)
        self.X.grad_ = Z.grad_ @ self.W
        return self.X.grad_

class ReLU():
    def forward(self, X):
        self.X = X
        return X.clamp_min(0.)

    def backward(self, A):
        return A.grad_ * (self.X > 0).float()

class SoftmaxCrossEntropy:
    def forward(self, X, y):
        e_x = torch.exp(X - torch.max(X, dim=-1, keepdim=True)[0])
        self.softmax_out = e_x / (torch.sum(e_x, dim=-1, keepdim=True) + 1e-10)

        log_probs = torch.log(self.softmax_out + 1e-10)
        target_log_probs = log_probs * y

        self.loss = -target_log_probs.sum(dim=-1).mean()
        return self.loss

    def backward(self, y):
        return (self.softmax_out - y) / y.shape[0]

class Model:
    def __init__(self, input_features, hidden_units, output_units, data_num):
        self.linear1 = Linear(input_features, hidden_units, data_num)
        self.relu = ReLU()
        self.linear2 = Linear(hidden_units, output_units, data_num)
        self.loss_fn = SoftmaxCrossEntropy()

    def forward(self, X, y):
        self.X = X
        self.Z1 = self.linear1.forward(X)
        self.A1 = self.relu.forward(self.Z1)
        self.Z2 = self.linear2.forward(self.A1)
        self.loss = self.loss_fn.forward(self.Z2, y)
        return self.loss, self.Z2

    def backward(self, y):
        self.Z2.grad_ = self.loss_fn.backward(y)
        self.A1.grad_ = self.linear2.backward(self.Z2)
        self.Z1.grad_ = self.relu.backward(self.A1)
        self.X.grad_ = self.linear1.backward(self.Z1)

    def zero_grad(self):
        # 勾配の初期化
        self.linear1.W.grad_ = None
        self.linear1.b.grad_ = None
        self.linear2.W.grad_ = None
        self.linear2.b.grad_ = None

    def step(self, learning_rate):
        # パラメータの更新
        self.linear1.W -= learning_rate * self.linear1.W.grad_
        self.linear1.b -= learning_rate * self.linear1.b.grad_
        self.linear2.W -= learning_rate * self.linear2.W.grad_
        self.linear2.b -= learning_rate * self.linear2.b.grad_

## Refactoring後の学習ループ(OptimizerやDataset, Dataloaderは後ほどRefactoring)
# ===データの準備====
dataset = datasets.load_digits()
data = dataset['data']
target = dataset['target']
images = dataset['images']
X_train, X_val, y_train, y_val = train_test_split(images, target, test_size=0.2, random_state=42)
X_train_mean = X_train.mean()
X_train_std = X_train.std()
X_train = (X_train - X_train_mean) / X_train_std
X_val = (X_val - X_train_mean) / X_train_std
X_train = torch.tensor(X_train.reshape(-1, 64), dtype=torch.float32).to(device)
X_val = torch.tensor(X_val.reshape(-1, 64), dtype=torch.float32).to(device)
y_train = F.one_hot(torch.tensor(y_train), num_classes=10).to(device) #1437 x 10
y_val = F.one_hot(torch.tensor(y_val), num_classes=10).to(device) # 360 x 10
batch_size = 30
# モデルの初期化
model = Model(input_features=64, hidden_units=1000, output_units=10, data_num=batch_size)

learning_rate = 0.01

# ログ
train_losses = []
val_losses = []
val_accuracies = []
for epoch in range(100):
    # エポック毎にデータをシャッフル
    shuffled_indices = np.random.permutation(len(y_train))
    num_batches = np.ceil(len(y_train)/batch_size).astype(int)
    running_loss = 0.0
    start_time = time.time()

    for i in range(num_batches):

        # mini batch作成
        start = i * batch_size
        end = start + batch_size

        batch_indices = shuffled_indices[start:end]
        y_true_ = y_train[batch_indices, :] # batch_size x 10

        X = X_train[batch_indices] # batch_size x 64
        # 順伝播と逆伝播の計算
        loss, _ = model.forward(X, y_true_)
        model.backward(y_true_)
        running_loss += loss.item()

        # パラメータ更新
        with torch.no_grad():
            model.step(learning_rate)

        model.zero_grad()

    # validation
    with torch.no_grad():
        val_loss, Z2_val = model.forward(X_val, y_val)

        val_accuracy = torch.sum(torch.argmax(Z2_val, dim=-1) == torch.argmax(y_val, dim=-1)) / y_val.shape[0]

    train_losses.append(running_loss/num_batches)
    val_losses.append(val_loss.item())
    val_accuracies.append(val_accuracy)
    end_time = time.time()
    epoch_duration = end_time - start_time
    print(f'epoch: {epoch}: train error: {running_loss/num_batches}, validation error: {val_loss.item()}, validation accuracy: {val_accuracy}, epoch duration {epoch_duration:.2f} sec')

epoch: 0: train error: 3.0376185284306607, validation error: 0.978164792060852, validation accuracy: 0.7944444417953491, epoch duration 0.05 sec
epoch: 1: train error: 0.7183706608678525, validation error: 0.5727800130844116, validation accuracy: 0.875, epoch duration 0.05 sec
epoch: 2: train error: 0.39205272775143385, validation error: 0.43261265754699707, validation accuracy: 0.8888888955116272, epoch duration 0.05 sec
epoch: 3: train error: 0.24988523579668254, validation error: 0.36632826924324036, validation accuracy: 0.9027777910232544, epoch duration 0.05 sec
epoch: 4: train error: 0.18320447819617888, validation error: 0.34293049573898315, validation accuracy: 0.9166666865348816, epoch duration 0.05 sec
epoch: 5: train error: 0.14127676023053937, validation error: 0.2696492671966553, validation accuracy: 0.9222222566604614, epoch duration 0.05 sec
epoch: 6: train error: 0.11387003109363529, validation error: 0.2497757375240326, validation accuracy: 0.9416666626930237, epoch du